In [1]:
import os
import requests
import re
from datetime import datetime

# URL to scrape for matching URLs
url_to_scrape = "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=6&ssid=25&smid=0%27"

# Send a GET request to the URL
response = requests.get(url_to_scrape)

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find all anchor (<a>) tags with href attributes
all_links = soup.find_all("a", href=True)

# Extract URLs that contain the specified pattern
pattern = "https://www.sebi.gov.in/media-and-notifications/public-notices/may-2024/"
matching_urls = [link["href"] for link in all_links if pattern in link["href"]]

# Create a directory to save the PDFs
folder_name = "attempt1"
os.makedirs(folder_name, exist_ok=True)

# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Now use the matching URLs to fetch PDF links
for url in matching_urls:
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Extract PDF links using regular expression
    pdf_links = re.findall(r'(?<=\.\./\.\./web/\?file=)https?://[^\s]+\.pdf', response.text)
    
    # Download PDFs and save them in the folder
    for pdf_link in pdf_links:
        # Get the PDF content
        pdf_response = requests.get(pdf_link)
        if pdf_response.status_code == 200:
            # Extract the file name from the URL
            file_name = pdf_link.split("/")[-1]
            # Rename the file by appending the current date
            renamed_file_name = f"{current_date}_{file_name}"
            # Save the PDF in the folder
            with open(os.path.join(folder_name, renamed_file_name), "wb") as file:
                file.write(pdf_response.content)
            print(f"PDF '{renamed_file_name}' downloaded successfully.")
        else:
            print(f"Failed to download PDF from '{pdf_link}'")
